In [ ]:
!nvidia-smi

Sat Jun  1 14:23:13 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  On   | 00000000:86:00.0 Off |                  Off |
| N/A   30C    P0    26W / 150W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  On   | 00000000:D8:00.0 Off |                  Off |
| N/A   

In [2]:
import gc
import os

import sys
#from fancy_einsum import einsum

import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookPoint,
)  # Hooking utilities
from transformers import AutoTokenizer, pipeline, logging, AutoModelForCausalLM, AutoConfig
from transformer_lens import ActivationCache, HookedTransformer

from huggingface_hub import notebook_login
from datasets import Dataset
import pandas as pd
import transformers
import torch
import einops  # Make sure einops is imported
import numpy as np 

from neel_plotly import line, imshow, scatter
import transformer_lens.patching as patching
import circuitsvis as cv
import matplotlib.pyplot as plt
from pathlib import Path


from utils.load_Datasets import load_Datasets
from utils.load_Model import load_Model
from utils.utils import *
import argparse


ModuleNotFoundError: No module named 'transformer_lens'

In [15]:
model,tokenizer = load_Model('olmo-7b',checkpoint="main")
model.cuda()

Trying to load model : olmo-1b / checkpoint : main -> from ../Models/OLMo-1B/main


OLMoForCausalLM(
  (model): OLMo(
    (transformer): ModuleDict(
      (wte): Embedding(50304, 2048)
      (emb_drop): Dropout(p=0.0, inplace=False)
      (ln_f): LayerNorm()
      (blocks): ModuleList(
        (0-15): 16 x OLMoSequentialBlock(
          (dropout): Dropout(p=0.0, inplace=False)
          (act): SwiGLU()
          (attn_out): Linear(in_features=2048, out_features=2048, bias=False)
          (ff_out): Linear(in_features=8192, out_features=2048, bias=False)
          (rotary_emb): RotaryEmbedding()
          (attn_norm): LayerNorm()
          (ff_norm): LayerNorm()
          (att_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (ff_proj): Linear(in_features=2048, out_features=16384, bias=False)
        )
      )
    )
  )
)

In [14]:
from utils.load_Datasets import load_Datasets
from utils.load_Model import load_Model
from utils.utils import *
if False:
    dataset_config_dict={
        'name':'openwebtext_parquet',
        'subset':[0,1],
        'base_directory':None
    }

    dataset=load_Datasets(dataset_config_dict)
if False:
    dataset_config_dict={
        'name':'dolma_v1.6_sample',
        'subset':[0],
        'base_directory':None
    }

    dataset=load_Datasets(dataset_config_dict)

if False:
    dataset_config_dict={
        'name':'pile_uncopyrighted',
        'subset':[0],
        'base_directory':None
    }

    dataset=load_Datasets(dataset_config_dict)

if True:
    dataset_config_dict={
        'name':'pile_uncopyrighted_parquet',
        'subset':[1],
        'base_directory':None
    }

    dataset=load_Datasets(dataset_config_dict)


In [16]:
len(dataset[1])

3

4278

["Biodegradability of the molluscicidal saponins of Phytolacca dodecandra.\nThe biodegradability of water-extracted saponins of berries from the Endod plant, Phytolacca dodecandra L'Herit, was evaluated under OECD standardized conditions. Persistence of the saponins was evaluated by determination of saponin concentrations in water over a 30-day period, using a quantitative HPLC method and a semiquantitative hemolytic assay, which is considered a potential field method. The two methods were compared. Bioassays were simultaneously conducted using Biomphalaria glabrata fresh water snails to assess the molluscicidal potency over time. All experiments were carried out in water, which was chemically balanced to sustain snails, and to one set river water was added to the medium to mimic natural conditions. (1) Saponin concentrations in water treated with an aqueous extract of P. dodecandra cultivar E44 were stable for 2 days then rapidly decreased during the third and fourth day. Lethal conce

In [24]:
batch=dataset[4:10]

len_list=get_length()
df = pd.DataFrame({key: value for key, value in batch.items() if key != 'text'})

# Print the DataFrame
print(df)

file_path = os.path.join(model_path, f"metadata.csv")
# Append the DataFrame to the file
if not os.path.isfile(file_path):
    df.to_csv(file_path, index=False)
else:
    df.to_csv(file_path, mode='a', header=False, index=False)

                 meta   id
0  [PubMed Abstracts]  1-4
1     [StackExchange]  1-5
2        [HackerNews]  1-6
3           [Pile-CC]  1-7
4           [Pile-CC]  1-8
5     [StackExchange]  1-9


torch.Size([6, 2024])

In [53]:
def get_len_list(tokens):
    len_list=(tokens['input_ids']==tokenizer.pad_token_id).int().argmax(dim=1)
    return len_list.numpy()

tensor([467, 156, 445, 438, 563, 842])

In [60]:

np.maximum(np.arange(5),0)

array([0, 1, 2, 3, 4])

In [63]:

# Set context length and batch size
ctx_len = 1024
batch_size = 4

pos_list = np.array([0,127,255,383,511,639,767,895,1010,1011,1022,1023,-1,-2,-3,-4])
project_name="Test_1"
cache = {}
model_path = f"Data/{project_name}"  # Update this to your desired path

module_name_list=["attn_out", "ff_out", ""]
layer_idx_list=np.array([0,3,7,12,13,14,15])
module_inblock_sentence="model.transformer.blocks.{layer}.{module}"
module_inblock_list = [
    module_inblock_sentence.format(layer=layer, module=module)
    for layer in layer_idx_list for module in module_name_list
]
module_outblock_list=["model.transformer.ln_f"]

module_list = module_inblock_list + module_outblock_list

# Create the directory if it doesn't exist
os.makedirs(model_path, exist_ok=True)

def hook_fn(module, input, output):
    # Retrieve the full name from the context in which the hook was registered
    full_name = hook_fn.full_names.get(module, "Unknown")  
    
    if isinstance(output, tuple):
        output = output[0]

    cache[full_name] = {}
    for pos in pos_list:
        if pos>=0:
            cache[full_name][pos]=output.detach()[:,pos,:]  # Keep tensor on CUDA
        elif pos<0:
            tensor=output.detach()[:,pos,:]
            cache[full_name][pos]=tensor[torch.arange(tensor.size(0)), np.maximum(hook_fn.len_list+pos,0)]  # Keep tensor on CUDA


def register_hooks(model, module_list, pos_list):
    hooks = []
    hook_fn.full_names = {}  # Dictionary to store full names accessible by module object
    hook_fn.pos_list=pos_list
    for name, module in model.named_modules():
        if name in module_list:
            hooks.append(module.register_forward_hook(hook_fn))
            hook_fn.full_names[module] = name
    return hooks

def remove_hooks(hooks):
    # Remove hooks after use
    for hook in hooks:
        hook.remove()
    return None

def get_len_list(tokens):
    len_list=(tokens['input_ids']==tokenizer.pad_token_id).int().argmax(dim=1)
    return len_list.numpy()

def save_cache_to_files(cache, model_path):
    for full_name, pos_dict in cache.items():
        for pos, tensor in pos_dict.items():
            file_path = os.path.join(model_path, f"{full_name.replace('.', '_')}_{pos}.pt")
            with open(file_path, 'ab') as f:  # Open file in append binary mode
                tensor_bytes = tensor.cpu().numpy().tobytes()
                f.write(len(tensor_bytes).to_bytes(4, byteorder='big'))
                f.write(tensor_bytes)

def save_loss_to_files(loss, model_path):
    for pos in pos_list:
        file_path = os.path.join(model_path, f"CREloss_{pos}.pt")
        with open(file_path, 'ab') as f:  # Open file in append binary mode
            tensor_bytes = loss[:,pos].cpu().numpy().tobytes()
            f.write(len(tensor_bytes).to_bytes(4, byteorder='big'))
            f.write(tensor_bytes)

def save_metadata_to_files(df_metadata, model_path):
    file_path = os.path.join(model_path, f"metadata.csv")
    if not os.path.isfile(file_path):
        df_metadata.to_csv(file_path, index=False)
    else:
        df_metadata.to_csv(file_path, mode='a', header=False, index=False)

def load_cache_from_file(file_path, dtype=torch.float32):
    tensors = []
    with open(file_path, 'rb') as f:
        while True:
            # Read the length of the next tensor_bytes
            len_bytes = f.read(4)
            if not len_bytes:
                break
            length = int.from_bytes(len_bytes, byteorder='big')
            # Read the actual tensor bytes
            tensor_bytes = f.read(length)
            # Convert bytes back to a numpy array
            tensor_array = np.frombuffer(tensor_bytes, dtype=np.float32)
            # Convert numpy array to a torch tensor
            tensor = torch.from_numpy(tensor_array)
            tensors.append(tensor)
    return tensors

def clear_gpu_memory(cache):
    for key in list(cache.keys()):
        del cache[key]
    torch.cuda.empty_cache()

# Function to process each batch and run the model
def run_batch_with_cache(batch):
    prompts = batch['text']
    tokens = tokenizer(prompts, padding='max_length', truncation=True, max_length=ctx_len, return_tensors='pt')
    input_ids = tokens['input_ids']
    attention_mask = tokens['attention_mask']

    len_list=get_len_list(tokens)
    df_metadata=pd.DataFrame({key: value for key, value in batch.items() if key != 'text'})
    hook_fn.len_list=len_list
    df_metadata['len']=len_list
    
    # Move tensors to the GPU if available
    if torch.cuda.is_available():
        input_ids = input_ids.cuda()
        attention_mask = attention_mask.cuda()

    # Run the model
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    
    # Get the logits
    logits = outputs.logits  # Shape: (batch_size, sequence_length, vocab_size)
    
    # Shift logits and input_ids for next token prediction
    shift_logits = logits[:, :-1, :].contiguous()
    shift_labels = input_ids[:, 1:].contiguous()
    
    # Flatten the tokens
    loss_fn = torch.nn.CrossEntropyLoss(reduction='none')
    loss = loss_fn(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    loss = loss.view(shift_logits.size(0), shift_logits.size(1))

    
    return df_metadata, outputs, loss.detach()


# Register hooks
hooks = register_hooks(model, module_list, pos_list)

# Iterate over the dataset with batch size of 4
for i in range(0, 10, batch_size):
    batch = dataset[i:i + batch_size]
    df_metadata, _, loss=run_batch_with_cache(batch)
    save_loss_to_files(loss, model_path)

    # Save cache to files
    save_cache_to_files(cache, model_path, len_list)
    save_metadata_to_files(df_metadata, model_path, len_list)
    save_loss_to_files(loss, model_path)
    print(cache)
    # Clear GPU memory
    if i>5:
        break
clear_gpu_memory(cache)

# Remove hooks after processing
remove_hooks(hooks)






IndexError: index 1023 is out of bounds for dimension 1 with size 1023

In [22]:
clear_gpu_memory(cache)

# Remove hooks after processing
remove_hooks(hooks)

NameError: name 'index' is not defined

5

In [12]:
def hook_fn(module, input, output):
    # Retrieve the full name from the context in which the hook was registered
    full_name = hook_fn.full_names.get(module, "Unknown")  
    print(output)
    if isinstance(output, tuple):
        output = output[0]

    if full_name not in cache:
        cache[full_name] = output.detach()  # Keep tensor on CUDA
    else:
        # Concatenate new output tensor along axis=1 with previous tensors, keep on CUDA
        cache[full_name] = torch.cat((cache[full_name], output.detach()), dim=1)


(tensor([[[-0.2052, -0.1213, -0.4657,  ..., -0.3354,  0.2140,  0.2558],
         [ 0.0714, -0.0663, -0.4005,  ..., -0.1806,  0.1556,  0.0736],
         [ 0.0319,  0.1940,  0.1250,  ..., -0.0757,  0.1653,  0.1655],
         ...,
         [ 0.1858,  0.0343, -0.0240,  ...,  0.0332, -0.2948, -0.1191],
         [ 0.1782,  0.1388,  0.0146,  ...,  0.1236, -0.3686, -0.2170],
         [ 0.1617,  0.2820, -0.0203,  ...,  0.0471, -0.3250, -0.1509]],

        [[-0.2528, -0.2245, -0.2554,  ..., -0.1681, -0.1213,  0.2140],
         [ 0.0865, -0.1196, -0.1393,  ..., -0.1530, -0.1192,  0.3871],
         [-0.0352,  0.0090, -0.4809,  ..., -0.2107, -0.2717,  0.2062],
         ...,
         [-0.2067,  0.3385, -0.4091,  ..., -0.3693, -0.0376,  0.2561],
         [ 0.0135, -0.0486, -0.2750,  ..., -0.0366, -0.2582,  0.0219],
         [-0.1556, -0.2162, -0.4740,  ..., -0.0490, -0.2857,  0.0990]],

        [[-0.4116, -0.2686, -0.3423,  ..., -0.4049,  0.0215,  0.2934],
         [-0.2542, -0.3633, -0.4045,  ..., -

AttributeError: 'Tensor' object has no attribute 'items'

In [15]:
model.model.transformer.blocks[0]

OLMoSequentialBlock(
  (dropout): Dropout(p=0.0, inplace=False)
  (act): SwiGLU()
  (attn_out): Linear(in_features=2048, out_features=2048, bias=False)
  (ff_out): Linear(in_features=8192, out_features=2048, bias=False)
  (rotary_emb): RotaryEmbedding()
  (attn_norm): LayerNorm()
  (ff_norm): LayerNorm()
  (att_proj): Linear(in_features=2048, out_features=6144, bias=False)
  (ff_proj): Linear(in_features=2048, out_features=16384, bias=False)
)

In [19]:
cache['model.transformer.blocks.15'].shape

torch.Size([4, 1024, 2048])

In [10]:
# Define the file path
file_path = "/home/gridsan/jsong/ML/ID_scaling/Data/Test_1/model_transformer_blocks_15.pt"

# Load the tensor from the file
tensor = load_cache_from_file(file_path)

tensor[0].shape

torch.Size([8388608])

In [8]:
cache.keys(), cache['model.transformer.blocks.15.attn_norm'].shape, cache['model.transformer.blocks.15.ff_proj'].shape

KeyError: 'model.transformer.blocks.15.attn_norm'

In [16]:
os.makedirs(model_path, exist_ok=True)


In [ ]:
                      
cache = {}

def hook_fn(module, input, output):
    # Retrieve the full name from the context in which the hook was registered
    full_name = hook_fn.full_names.get(module, "Unknown")  
    if full_name not in cache:
        cache[full_name] = output.detach()  # Keep tensor on CUDA
    else:
        # Concatenate new output tensor along axis=1 with previous tensors, keep on CUDA
        cache[full_name] = torch.cat((cache[full_name], output.detach()), dim=1)

def register_hooks(model, module_list):
    hooks = []
    hook_fn.full_names = {}  # Dictionary to store full names accessible by module object
    for name, module in model.named_modules():
        if name in module_list:
            hooks.append(module.register_forward_hook(hook_fn))
            hook_fn.full_names[module] = name
    return hooks

# Setup for model and tokenizer
module_list = ["model.transformer.blocks.15.attn_norm", "model.transformer.blocks.15.ff_proj"]
hooks = register_hooks(model, module_list)

# Example text generation
message = ["Language modeling is "]
inputs = tokenizer(message, return_tensors='pt', return_token_type_ids=False).to(device)
with torch.no_grad():
    response = model.generate(**inputs, max_new_tokens=40, do_sample=True, top_k=50, top_p=0.95)
    print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

# Remove hooks after use
for hook in hooks:
    hook.remove()

# Move to CPU after all GPU computations if needed
for key in cache:
    cache[key] = cache[key].cpu()  # Now move tensors to CPU


tensor([[26170, 14053,   310,   209, 50277, 50279]])

In [ ]:
os.listdir(os.path.expanduser("~")+'/ML/Models/OLMo-1B/'), os.listdir(os.path.expanduser("~")+'/ML/Models/OLMo-7B/')

(['step20000-tokens84B',
  'step600000-tokens2517B',
  'step50000-tokens210B',
  'step350000-tokens1468B',
  'step550000-tokens2307B',
  'step700000-tokens2936B',
  'step450000-tokens1887B',
  '.ipynb_checkpoints',
  'step400000-tokens1678B',
  'step500000-tokens2097B',
  '.git',
  'step100000-tokens419B',
  '.gitattributes',
  'step650000-tokens2726B',
  'main'],
 ['step450000-tokens1991B',
  'step350000-tokens1548B',
  'step250000-tokens1106B',
  'step50000-tokens221B',
  'step20000-tokens88B',
  'step400000-tokens1769B',
  'step150000-tokens664B',
  'step30000-tokens133B',
  'step300000-tokens1327B',
  'step550000-tokens2433B',
  '.ipynb_checkpoints',
  'step10000-tokens44B',
  '.git',
  'step500000-tokens2212B',
  'step200000-tokens885B',
  'step40000-tokens177B',
  '.gitattributes',
  'main',
  'step100000-tokens442B'])

In [ ]:


model_path=os.path.expanduser("~")+'/ML/Models/OLMo-1B/'+'step600000-tokens2517B'
config_path=model_path+'/config.json'
config=AutoConfig.from_pretrained(config_path)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True)
for param in model.parameters():
    param.requires_grad = False
model=model.to(torch.device("cuda"))
model.eval()


OLMoForCausalLM(
  (model): OLMo(
    (transformer): ModuleDict(
      (wte): Embedding(50304, 2048)
      (emb_drop): Dropout(p=0.0, inplace=False)
      (ln_f): LayerNorm()
      (blocks): ModuleList(
        (0-15): 16 x OLMoSequentialBlock(
          (dropout): Dropout(p=0.0, inplace=False)
          (act): SwiGLU()
          (attn_out): Linear(in_features=2048, out_features=2048, bias=False)
          (ff_out): Linear(in_features=8192, out_features=2048, bias=False)
          (rotary_emb): RotaryEmbedding()
          (attn_norm): LayerNorm()
          (ff_norm): LayerNorm()
          (att_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (ff_proj): Linear(in_features=2048, out_features=16384, bias=False)
        )
      )
    )
  )
)

In [ ]:
hook_outputs = {}

def hook_fn(module, input, output):
    #print(f"Hook on {module.__class__.__name__}")
    hook_outputs[module] = output
    return output

# Register hooks on the specified layers
def register_hooks(model):
    hooks = []
    layers = model.model.transformer.blocks
    
    # Register hooks on the 2nd, 3rd, and last layers
    layers_to_hook = [1, 2, len(layers) - 1]
    
    for layer_idx in layers_to_hook:
        layer = layers[layer_idx]
        
        # Register hooks on the feed-forward network output (ff_out)
        hooks.append(layer.ff_out.register_forward_hook(hook_fn))
    
    return hooks

model=olmo

hooks = register_hooks(model)

message = ["Language modeling is "]
inputs = tokenizer(message, return_tensors='pt', return_token_type_ids=False).to(torch.device("cuda"))
with torch.no_grad():
    response = model.generate(**inputs, max_new_tokens=40, do_sample=True, top_k=50, top_p=0.95)
    print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])
for hook in hooks:
    hook.remove()

Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Linear
Hook on Li

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

OLMoForCausalLM(
  (model): OLMo(
    (transformer): ModuleDict(
      (wte): Embedding(50304, 2048)
      (emb_drop): Dropout(p=0.0, inplace=False)
      (ln_f): LayerNorm()
      (blocks): ModuleList(
        (0-15): 16 x OLMoSequentialBlock(
          (dropout): Dropout(p=0.0, inplace=False)
          (act): SwiGLU()
          (attn_out): Linear(in_features=2048, out_features=2048, bias=False)
          (ff_out): Linear(in_features=8192, out_features=2048, bias=False)
          (rotary_emb): RotaryEmbedding()
          (attn_norm): LayerNorm()
          (ff_norm): LayerNorm()
          (att_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (ff_proj): Linear(in_features=2048, out_features=16384, bias=False)
        )
      )
    )
  )
)

In [ ]:
                      
cache = {}

def hook_fn(module, input, output):
    # Retrieve the full name from the context in which the hook was registered
    full_name = hook_fn.full_names.get(module, "Unknown")  
    if full_name not in cache:
        cache[full_name] = output.detach()  # Keep tensor on CUDA
    else:
        # Concatenate new output tensor along axis=1 with previous tensors, keep on CUDA
        cache[full_name] = torch.cat((cache[full_name], output.detach()), dim=1)

def register_hooks(model, module_list):
    hooks = []
    hook_fn.full_names = {}  # Dictionary to store full names accessible by module object
    for name, module in model.named_modules():
        if name in module_list:
            hooks.append(module.register_forward_hook(hook_fn))
            hook_fn.full_names[module] = name
    return hooks

# Setup for model and tokenizer
model = olmo  # Ensure model is correctly assigned
module_list = ["model.transformer.blocks.15.attn_norm", "model.transformer.blocks.15.ff_proj"]
hooks = register_hooks(model, module_list)

# Example text generation
message = ["Language modeling is "]
inputs = tokenizer(message, return_tensors='pt', return_token_type_ids=False).to(device)
with torch.no_grad():
    response = model.generate(**inputs, max_new_tokens=40, do_sample=True, top_k=50, top_p=0.95)
    print(tokenizer.batch_decode(response, skip_special_tokens=True)[0])

# Remove hooks after use
for hook in hooks:
    hook.remove()

# Move to CPU after all GPU computations if needed
for key in cache:
    cache[key] = cache[key].cpu()  # Now move tensors to CPU


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument tensors in method wrapper_CUDA_cat)

In [ ]:
model()

OLMoForCausalLM(
  (model): OLMo(
    (transformer): ModuleDict(
      (wte): Embedding(50304, 2048)
      (emb_drop): Dropout(p=0.0, inplace=False)
      (ln_f): LayerNorm()
      (blocks): ModuleList(
        (0-15): 16 x OLMoSequentialBlock(
          (dropout): Dropout(p=0.0, inplace=False)
          (act): SwiGLU()
          (attn_out): Linear(in_features=2048, out_features=2048, bias=False)
          (ff_out): Linear(in_features=8192, out_features=2048, bias=False)
          (rotary_emb): RotaryEmbedding()
          (attn_norm): LayerNorm()
          (ff_norm): LayerNorm()
          (att_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (ff_proj): Linear(in_features=2048, out_features=16384, bias=False)
        )
      )
    )
  )
)

In [ ]:

tensor = cache['model.transformer.blocks.15.attn_norm']
if tensor.is_cuda:
    print("Tensor is on CUDA device:", tensor.get_device())
else:
    print("Tensor is on CPU")

Tensor is on CPU
